# Griffin-Lim法に基づく位相復元

In [1]:
import numpy as np
from scipy.io import wavfile
import librosa
from pypesq import pesq
from IPython.display import Audio

In [2]:
IN_WAVE_FILE = "in.wav"  # モノラル音声
OUT_WAVE_FILE = "out_admm_gla.wav"  # 復元音声

In [3]:
FRAME_LENGTH = 1024        # フレーム長 (FFTサイズ)
HOP_LENGTH = 80            # フレームのシフト長
ITERATION = 100            # 位相推定の最大繰り返し数


In [4]:
# 音声のロード
fs, data = wavfile.read(IN_WAVE_FILE)
data = data.astype(np.float64)

## 振幅スペクトル（位相復元なので手に入るのはこれのみ）

In [5]:
# 振幅スペクトル（位相復元なので手に入るのはこれのみ）
amp_spec = np.abs(
    librosa.core.stft(
        data, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH
    )
)

In [6]:
# 乱数の種を指定して再現性を保証
np.random.seed(seed=0)

## Griffin-Lim法による位相復元

In [7]:
# Griffin-Lim法に基づく位相スペクトルの推定
for i in range(ITERATION):
    if i == 0:
        # 初回は乱数で初期化
        phase_spec = np.random.rand(*amp_spec.shape)
    else:
        # 振幅スペクトルと推定された位相スペクトルから複素スペクトログラムを復元
        recovered_spec = amp_spec * np.exp(1j * phase_spec)

        # 短時間フーリエ逆変換で音声を復元
        recovered = librosa.core.istft(recovered_spec, hop_length=HOP_LENGTH,
                                       win_length=FRAME_LENGTH)

        # 復元音声から複素スペクトログラムを再計算
        complex_spec = librosa.core.stft(recovered, n_fft=FRAME_LENGTH,
                                         hop_length=HOP_LENGTH,
                                         win_length=FRAME_LENGTH)

        # 初回以降は計算済みの複素スペクトログラムから位相スペクトルを推定
        phase_spec = np.angle(complex_spec)

In [8]:
# 音声を復元
recovered_spec = amp_spec * np.exp(1j * phase_spec)
recovered_gla = librosa.core.istft(recovered_spec, hop_length=HOP_LENGTH,
                               win_length=FRAME_LENGTH)
# pesqで音質を評価
print("PESQ = ", pesq(recovered_gla, data[: len(recovered_gla)], fs))

PESQ =  4.051231384277344


## 音声の聴き比べ

### 元音声

In [9]:
Audio(data, rate=fs)

### Griffin-Lim法による復元

In [10]:
Audio(recovered_gla, rate=fs)